## Deriving the Weak Form

$\frac{d^2 u}{dx^2} + u - x^2 = 0$

Taking from this, we setup:

$\int_a^b w(\frac{d^2 u}{dx^2} + u) dx = \int_a^b x^2 w dx$

where w and u are both functions of x. Using integration by parts we obtain

$w \frac{du}{dx} \bigg|_a^b - \int_a^b(\frac{dw}{dx} \frac{du}{dx} + wu) dx = \int_a^b x^2 w dx$

Now we have $B(w,u) = w \frac{du}{dx} \bigg|_a^b - \int_a^b(\frac{dw}{dx} \frac{du}{dx} + wu) dx$ and $\ell(w) = \int_a^b x^2 w dx$

## Converting to N

$u(x) \approx \sum_{j=1}^2 u_j N_j(x)$ where $x \in [x_i, x_{i+1}]$

$N_1(x) = \frac{x_{i+1}-x}{h_e}$, and $N_2(x) = \frac{x - x_i}{h_e}$, where $h_e = \frac{1}{m} = x_{i+1} - x_i$

$s = \frac{1}{h_e} (2x-(x_i + x_{i+1})) \ \rightarrow \ x = \frac{1}{2} (h_e s + (x_i + x_{i+1}))$

$ds = \frac{2}{h_e} dx$, and $dx = \frac{h_e}{2} ds$

$N_1(s) = \frac{1}{2} (1-s)$, $N_2(s) = \frac{1}{2}(1+s)$; $\frac{dN_1}{ds} = -\frac{1}{2}$, and $\frac{dN_2}{ds} = \frac{1}{2}$

$\frac{dN_1}{dx} = - \frac{1}{2} \frac{2}{h_e}$, $\frac{dN_2}{dx} = \frac{1}{2} \frac{2}{h_e}$

$\longrightarrow \ \sum_{j=1}^2 k_{ij} u_j = f_i \ \Rightarrow \ k_{ij} = N_i \frac{dN_j}{dx} \bigg|_{x_i}^{x_{i+1}}
- \int_{x_i}^{x_{i+1}} \big(N_i N_j + \frac{dN_i}{dx} \frac{dN_j}{dx}\big) dx$ and $f_i = \int_{x_i}^{x_{i+1}} x^2 N_i dx$

### $ x \ \rightarrow \ s$
$k_{ij} = N_i \frac{2}{h_e} \frac{dN_j}{ds} \bigg|_{-1}^1 - \int_{-1}^1 \big(N_i N_j + \frac{4}{h_e^2} \frac{dN_i}{ds} \frac{dN_j}{ds}\big) \frac{h_e}{2} ds$

$\beta_i = \int_{-1}^1 \frac{1}{4} (h_e s + (x_i + x_{i+1}))^2 N_i^2 \frac{h_e}{2} ds$ Here I switched from $b$ to $\beta$ to avoid confusion with the value of the upper bound of the system (where $x \in [a,b]$)

In [30]:
# Plotting packages
using Makie
using AbstractPlotting
using WGLMakie
WGLMakie.activate!()

# Guassian Quadrature package to compute integrals
using QuadGK

# Used for math
using LinearAlgebra


#=
This program is written to solve the second-order differential equation:
    u" + u - x² = 0
for x ∈ [0,1]
=#

In [75]:
# ==========
# Input parameters
a = 0;  # Lower bound
b = 1;  # Upper bound
k = 2;  # Number of nodes
m = 4;  # Number of elements

# Boundary conditions
    ## Essential (Dirichlet) boundary conditions
    xa = 1/2;   # Lower essential boundary condition at u(a)
    xb = 1;     # Upper essential boundary condition at u(b)

    essential_a = 1;    # 1 for lower essential true, 0 for false.
    essential_ab = 0;   # Both essential conditions true

    ## Natural (Neumann) boundary conditions
    qa = 0;    # Lower natural boundary condition at u'(a)
    qb = 4/3;   # Upper natural boundary condition at u'(b)
    qᵥ = zeros(m+1);
    qᵥ[begin] = -qa;
    qᵥ[end] = qb;

# ==========

In [76]:
hₑ = 1/m;   # Step size

global u = zeros(m+1);
global K = zeros(m+1, m+1);
global β = zeros(m+1);

global xᵥ = a:hₑ:b; # array of xᵢ node points

function N(ℸ, s)
    if ℸ == 1
        (1 - s) / hₑ;
    elseif ℸ == 2
        (1 + s)   / hₑ;
    end
end

dN = [-(1/2), (1/2)];

In [77]:
# ==========
# Solving along each 2-node element

for ℵ ∈ 1:m     # Indicie of node, essentially the i for xᵢ
    for ℶ ∈ 1:k
        for ℷ ∈ 1:k
            k₁ = ( (2/hₑ) * N(ℶ, 1) * dN[ℷ] ) - ( (2/hₑ) * N(ℶ, -1) * dN[ℷ] );
            # Finds (2/hₑ) Nᵢ dNⱼ/ds from -1 to 1

            k₂(s) = ( (N(ℶ, s) * N(ℷ, s)) + (4/(hₑ^2)) * dN[ℶ] * dN[ℷ] ) * (hₑ/2);
            ktemp, err = quadgk(s -> k₂(s), -1, 1);
            # Solves the integral ∫(he/2)(NᵢNⱼ + (4/hₑ^2) dNᵢ/ds dNⱼ/ds) ds from -1 to -1
            # Computes this using a Gaussian quadrature package

            K[ℵ+(ℶ-1),ℵ+(ℷ-1)] += (k₁ + ktemp); # Sum the above to get Kᵢⱼ
        end
    end

    β_integrand_1(s) = (1/4) * (hₑ*s + (xᵥ[ℵ] + xᵥ[ℵ+1]))^2 * N(1, s) * (hₑ/2);
    βtemp_1, err = quadgk(s -> β_integrand_1(s), -1, 1);
    # Computes the integral ∫ (1/4)(hₑs + (xᵢ+xᵢ₊₁))^2 N₁(hₑ/2)ds from -1 to 1

    β[ℵ] += βtemp_1;

    β_integrand_2(s) = (1/4) * (hₑ*s + (xᵥ[ℵ] + xᵥ[ℵ+1]))^2 * N(2, s) * (hₑ/2);
    βtemp_2, err = quadgk(s -> β_integrand_2(s), -1, 1);
    # Computes the integral ∫ (1/4)(hₑs + (xᵢ+xᵢ₊₁))^2 N₂(hₑ/2)ds from -1 to 1

    β[ℵ+1] = βtemp_2;
end

In [20]:
println(K)

[12.666666666666668 -8.666666666666666 0.0; -8.666666666666666 25.333333333333336 -8.666666666666666; 0.0 -8.666666666666666 12.666666666666668]


In [21]:
println(β)

[0.04166666666666667, 0.5833333333333334, 0.7083333333333333]


In [78]:
# ==========
# Time to set essential boundary conditions
if essential_a == 1
    K[begin, :] .= 0;
    K[:, begin] .= 0;
    K[begin, begin] = xa;
    β[begin] = xa;
elseif essential_ab == 1
    K[begin, :] .= 0;
    K[:, begin] .= 0;
    K[begin, begin] = xa;
    β[begin] = xa;
    # ----------
    K[end, :] .= 0;
    K[:, end] .= 0;
    K[end, end] = xb;
    β[end] = xb;
end

f = qᵥ .+ β;

u = K \ f;

In [23]:
#Array used for exact solution plot

xs = a:0.001:b;
lxs = length(xs);
u_exact_array = zeros(lxs);

## 1. Specified essential (Dirichlet) boundary conditions:

$u(x=0) = \frac{1}{2}$, $u(x=1) = 1$

In [46]:
# Exact solution for 1.
uₑ₁(x) = -2 + x^2 + (5/2) * cos(x) - (1/2) * (5*cot(1) - 4*csc(1))*sin(x);

for i ∈ 1:lxs
    u_exact_array[i] = uₑ₁(xs[i]);
end

### (a) $m=2$

In [33]:
scene = lines(xs, u_exact_array, color = :black)
lines!(scene, a:hₑ:b, u, color = :red) 

### (b) $m = 3$

In [41]:
scene = lines(xs, u_exact_array, color = :black)
lines!(scene, a:hₑ:b, u, color = :red)

### (c) $m = 4$

In [47]:
scene = lines(xs, u_exact_array, color = :black)
lines!(scene, a:hₑ:b, u, color = :red)

## 2. Specified natural (Neumann) boundary conditions:

$\frac{d}{dx} u(x=0) = -1$, $\frac{d}{dx}u(x=1) = \frac{4}{3}$

In [52]:
# Exact solution for 2.
uₑ₂(x) = -2 + x^2 + (1/3) * (2*csc(1) - 3*cot(1))*cos(x) - sin(x);

for i ∈ 1:lxs
    u_exact_array[i] = uₑ₂(xs[i]);
end

### (a) $m = 2$

In [53]:
scene = lines(xs, u_exact_array, color = :black)
lines!(scene, a:hₑ:b, u, color = :red) 

### (b) $m = 3$

In [58]:
scene = lines(xs, u_exact_array, color = :black)
lines!(scene, a:hₑ:b, u, color = :red) 

### (c) $m = 4$

In [63]:
scene = lines(xs, u_exact_array, color = :black)
lines!(scene, a:hₑ:b, u, color = :red) 

## 3. Specified mixed (essential and natural) boundary conditions:

$u(x=0) = \frac{1}{2}$, $\frac{d}{dx} u(x=1) = \frac{4}{3}$

In [68]:
# Exact solution for 3.
uₑ₃(x) = -2 + x^2 + (5/2) *cos(x) - (1/6)*(4-15*sin(1))*sec(1)*sin(x);

for i ∈ 1:lxs
    u_exact_array[i] = uₑ₃(xs[i]);
end

### (a) $m = 2$

In [69]:
scene = lines(xs, u_exact_array, color = :black)
lines!(scene, a:hₑ:b, u, color = :red) 

### (b) $m = 3$

In [74]:
scene = lines(xs, u_exact_array, color = :black)
lines!(scene, a:hₑ:b, u, color = :red) 

### (c) $m = 4$

In [79]:
scene = lines(xs, u_exact_array, color = :black)
lines!(scene, a:hₑ:b, u, color = :red)

An exception was thrown in JS: TypeError: Cannot read property 'getContext' of null
Additional message: Error during running onjs callback
Callback:
function setup([scenes, duplicates]){
    const canvas = (document.querySelector('[data-jscall-id="25"]'))
    const renderer = WGLMakie.threejs_module(canvas, '2089797558891523662', 960, 540)
    WGLMakie.set_duplicate_references(duplicates)
    const three_scenes = scenes.map(WGLMakie.deserialize_scene)
    const cam = new THREE.PerspectiveCamera(45, 1, 0, 100)
    console.log(three_scenes)
    WGLMakie.start_renderloop(renderer, three_scenes, cam)

    function get_plot(plot_uuid) {
        for (const idx in three_scenes) {
            const plot = three_scenes[idx].getObjectByName(plot_uuid)
            if (plot) {
                return plot
            }
        }
        return undefined;
    }

    function add_plot(scene, plot) {
        const mesh = WGLMakie.deserialize_plot(plot);
    }
    const context = {
        three_scenes

### Discussion

The data from all of my runs does not match any of the exact solutions. Only for the essential boundary conditions for problem 1 did they actually hold. I am unsure of exactly what I am doing wrong, whether it is in the actual derivation at the top, or in the setup of the program itself. 